In [2]:
import os
import sys
from pathlib import Path

from genlm_control import InferenceEngine, PromptedLLM, BoolCFG, eager_token_sampler

genlm_control_path = Path.cwd().parent.parent
if str(genlm_control_path) not in sys.path:
    sys.path.insert(0, str(genlm_control_path))

from benchmark.text_to_sql.run_inference import spider_setup  # noqa: E402

In [3]:
raw_spider_dir = "data/spider_data"
model_name = "meta-llama/Llama-3.2-1B-Instruct"
lm_backend = "hf"
grammar_dir = "data/grammars"

In [4]:
dev_data, _, prompt_formatter = spider_setup(raw_spider_dir)

In [ ]:
llm = PromptedLLM.from_name(model_name, backend=lm_backend)

/opt/homebrew/Caskroom/miniconda/base/envs/genlm/lib/python3.11/site-packages/genlm_backend/tokenization/vocab.py:99: UserWarning: Duplicate tokens found in string vocabulary. This may lead to downstream issues with the string vocabulary; we recommend using the byte vocabulary.
  warnings.warn(


In [6]:
datum = dev_data[0]

llm.prompt_ids = llm.model.tokenizer.apply_chat_template(
    prompt_formatter.format_openai(datum),
    add_generation_prompt=True,
    tokenize=True,
)

In [7]:
grammar = open(os.path.join(grammar_dir, f"{datum.schema_name}.lark"), "r").read()
bool_cfg = BoolCFG.from_lark(grammar)

In [8]:
sampler = eager_token_sampler(llm, bool_cfg)

/opt/homebrew/Caskroom/miniconda/base/envs/genlm/lib/python3.11/site-packages/genlm_backend/trie/parallel.py:63: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  ).to_sparse_csr()


In [10]:
from genlm_control.experimental.vegas import GumbelMaxAdaptiveRejectionSampler

sampler = GumbelMaxAdaptiveRejectionSampler(llm, bool_cfg.coerce(llm, f=b"".join))

In [ ]:
datum = dev_data[0]

llm.prompt_ids = llm.model.tokenizer.apply_chat_template(
    prompt_formatter.format_openai(datum),
    add_generation_prompt=True,
    tokenize=True,
)

In [12]:
sequences = await InferenceEngine(sampler)(
    n_particles=3,
    max_tokens=100,
    ess_threshold=0.9,
    verbosity=1,
    # json_path=os.path.join('results/test', f'0.json')
)

-0.00:	[b'SELECT']
-0.00:	[b'SELECT']
-0.00:	[b'SELECT']
-0.00:	[b'SELECT'|b' COUNT']
-0.00:	[b'SELECT'|b' COUNT']
-0.00:	[b'SELECT'|b' COUNT']
-0.00:	[b'SELECT'|b' COUNT'|b'(S']
-0.00:	[b'SELECT'|b' COUNT'|b'(S']
-0.00:	[b'SELECT'|b' COUNT'|b'(S']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger'|b'_ID']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger'|b'_ID']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger'|b'_ID']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger'|b'_ID'|b')']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger'|b'_ID'|b')']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger'|b'_ID'|b')']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger'|b'_ID'|b')'|b' FROM']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger'|b'_ID'|b')'|b' FROM']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger'|b'_ID'|b')'|b' FROM']
-0.00:	[b'SELECT'|b' COUNT'|b'(S'|b'inger'|b'_ID'|b')'|b' FROM'|b' singer']
-0.00:	[b'SELECT'|b' COUNT'